# convert time

In [1]:
def clean_time_string(time_str):
    """
    Clean the time string by removing extraneous annotations like [note 1].
    Handles cases where the input is None, NaN, or non-string.
    """
    # Check if the input is not a string
    if not isinstance(time_str, str):
        return ""  # Return an empty string for invalid inputs
    
    # Remove any content inside brackets (e.g., [note 1])
    cleaned_time = re.sub(r'\[.*?\]', '', time_str)
    return cleaned_time.strip()


In [2]:
def convert_time(row):
    """
    Convert time to UTC and extract timezone information.
    Handles AM/PM, UTC offsets, timezone acronyms, and invalid inputs.
    """
    time_str = clean_time_string(row['time'])
    if not time_str:  # Check for empty or invalid time strings
        return None, None  # Return default values for invalid input
    
    try:
        # Step 1: Extract time (24-hour format) and normalize AM/PM
        match_time = re.search(r'(\d{1,2}:\d{2})', time_str)
        if not match_time:
            raise ValueError("No valid time found in string")
        
        local_time = match_time.group(1)  # Extract the time portion
        if "a.m." in time_str.lower():
            time_str = time_str.lower().replace("a.m.", "").strip()
        elif "p.m." in time_str.lower():
            time_str = time_str.lower().replace("p.m.", "").strip()
            hour, minute = map(int, local_time.split(":"))
            if hour != 12:  # Convert to 24-hour format for PM times
                hour += 12
            local_time = f"{hour:02}:{minute:02}"
        
        # Step 2: Extract UTC offset from parenthesis
        match_utc = re.search(r'\(UTC([+-]\d+)\)', time_str)
        utc_offset = int(match_utc.group(1)) if match_utc else 0
        
        # Step 3: Handle specific timezone acronyms
        timezone_offsets = {
            "CEST": 2, "CET": 1, "EEST": 3, "EDT": -4, "EST": -5,
            "PDT": -7, "PST": -8, "CDT": -5, "CST": -6, 
            "JST": 9, "KST": 9, "BRT": -3, "MSK": 3
        }
        for tz, offset in timezone_offsets.items():
            if tz in time_str:
                utc_offset = offset
                break

        # Step 4: Convert local time to UTC
        local_time_obj = datetime.strptime(local_time, '%H:%M')
        utc_time_obj = local_time_obj - timedelta(hours=utc_offset)
        
        return utc_time_obj.strftime('%H:%M'), f"UTC{utc_offset:+}"
    
    except Exception as e:
        print(f"Error processing time: {time_str}. Error: {e}")
        return None, None


# after first day league standings and points

In [3]:
def after_first(goals_df):
    """
    Process goals data to calculate standings after the first matchday 
    and create datasets for games in the last two matchdays.
    """

    # Step 1: Find the first match date for each tournament and group
    first_dates = goals_df.groupby(['year', 'stage'])['short_date'].min().reset_index()

    # Step 2: Create the dataset with all games for the first matchday (goals_first_matchday)
    goals_first_matchday = goals_df.merge(
        first_dates, on=['year', 'stage', 'short_date'], how='inner'
    )

    # Step 3: Create the dataset with all games excluding the first matchday (goals_after_first_matchday)
    goals_after_first_matchday = goals_df.merge(
        first_dates, on=['year', 'stage', 'short_date'], how='left', indicator=True
    )
    goals_after_first_matchday = goals_after_first_matchday[goals_after_first_matchday['_merge'] == 'left_only'].drop(columns=['_merge'])

    # Step 4: Convert short_date to datetime to ensure nlargest can be used
    goals_after_first_matchday['short_date'] = pd.to_datetime(goals_after_first_matchday['short_date'])

    # Find the last two match dates for each tournament and group
    last_two_dates = (
        goals_after_first_matchday.groupby(['year', 'stage'])['short_date']
        .nlargest(2)
        .reset_index()
    )

    # Step 5: Create datasets for the last two matchdays
    goals_last_two_matchdays = goals_after_first_matchday.merge(
        last_two_dates, on=['year', 'stage', 'short_date'], how='inner'
    )

    # Ensure that goals_last_two_matchdays is sorted properly by short_date and goal_minute
    goals_last_two_matchdays_sorted = goals_last_two_matchdays.sort_values(by=['short_date', 'goal_minute'], ascending=[True, True])

    # Initialize an empty list to store the results for games in the first matchday
    results = []

    # Iterate over each match in goals_first_matchday
    for match_id, group in goals_first_matchday.groupby(['year', 'stage', 'home_team', 'away_team']):
        # Initialize goals_home and goals_away for each match
        goals_home = 0
        goals_away = 0

        # Extract local_time and score from the first row in the group
        local_time = group.iloc[0]['local_time']
        score = group.iloc[0]['score']

        # Extract short_date from local_time (convert to date)
        short_date = pd.to_datetime(local_time).date()

        # Loop through each row to count goals for home and away teams
        for _, row in group.iterrows():
            if row['home_team'] == row['scorer_nationality']:
                goals_home += 1
            elif row['away_team'] == row['scorer_nationality']:
                goals_away += 1

        # Calculate the expected score and normalize both scores
        calculated_score = f"{goals_home}-{goals_away}"
        normalized_score = score.replace("–", "-").replace("—", "-")

        # Check if the normalized score matches the calculated score
        score_match = normalized_score == calculated_score

        # Append the results to the list
        results.append({
            'year': match_id[0],
            'stage': match_id[1],
            'home_team': match_id[2],
            'away_team': match_id[3],
            'local_time': local_time,
            'short_date': short_date,
            'goals_home': goals_home,
            'goals_away': goals_away,
            'original_score': score,
            'calculated_score': calculated_score,
            'score_match': score_match
        })

    # Convert results list into a DataFrame
    agg_goals_after_first_day = pd.DataFrame(results)

    # Add 'won' column based on comparison of goals_home and goals_away
    agg_goals_after_first_day['won'] = agg_goals_after_first_day.apply(
        lambda row: 1 if row['goals_home'] > row['goals_away'] else (-1 if row['goals_home'] < row['goals_away'] else 0), 
        axis=1
    )

    return agg_goals_after_first_day, goals_last_two_matchdays_sorted


# before last match day league standings and points

In [4]:
def before_last(goals_df):
    """
    Process goals data to create datasets for games before and on the last match day.
    Calculates home and away goals, and adds a 'won' column indicating match result.
    """
    
    # Step 1: Find the last match date for each tournament and group
    last_dates = goals_df.groupby(['year', 'stage'])['short_date'].max().reset_index()

    # Step 2: Create the dataset with all games excluding the last match day (goals_before_last_day)
    goals_before_last_day = goals_df.merge(
        last_dates, on=['year', 'stage', 'short_date'], how='left', indicator=True
    )
    goals_before_last_day = goals_before_last_day[goals_before_last_day['_merge'] == 'left_only'].drop(columns=['_merge'])

    # Step 3: Create the dataset with only the last match day games (goals_last_day)
    goals_last_day = goals_df.merge(last_dates, on=['year', 'stage', 'short_date'], how='inner')

    # Step 4: Sort goals_last_day by 'goal_minute'
    goals_last_day_sorted = goals_last_day.sort_values(by=['short_date', 'goal_minute'], ascending=True)

    # Ensure that goals_last_day_sorted has no duplicates
    goals_last_day_sorted = goals_last_day_sorted.drop_duplicates()

    # Initialize an empty list to store the results for games before the last day
    results = []

    # Iterate over each match in goals_before_last_day
    for match_id, group in goals_before_last_day.groupby(['year', 'stage', 'home_team', 'away_team']):
        # Initialize goals_home and goals_away for each match
        goals_home = 0
        goals_away = 0
        
        # Extract local_time and score from the first row in the group
        local_time = group.iloc[0]['local_time']
        score = group.iloc[0]['score']
        
        # Extract short_date from local_time (convert to date)
        short_date = pd.to_datetime(local_time).date()
        
        # Loop through each row to count goals for home and away teams
        for _, row in group.iterrows():
            if row['home_team'] == row['scorer_nationality']:
                goals_home += 1
            elif row['away_team'] == row['scorer_nationality']:
                goals_away += 1

        # Calculate the expected score and normalize both scores
        calculated_score = f"{goals_home}-{goals_away}"
        normalized_score = score.replace("–", "-").replace("—", "-")
        
        # Check if the normalized score matches the calculated score
        score_match = normalized_score == calculated_score

        # Append the results to the list
        results.append({
            'year': match_id[0],
            'stage': match_id[1],
            'home_team': match_id[2],
            'away_team': match_id[3],
            'local_time': local_time,
            'short_date': short_date,
            'goals_home': goals_home,
            'goals_away': goals_away,
            'original_score': score,
            'calculated_score': calculated_score,
            'score_match': score_match
        })

    # Convert results list into a DataFrame
    agg_goals_before_last_day = pd.DataFrame(results)

    # Add 'won' column based on comparison of goals_home and goals_away
    agg_goals_before_last_day['won'] = agg_goals_before_last_day.apply(
        lambda row: 1 if row['goals_home'] > row['goals_away'] else (-1 if row['goals_home'] < row['goals_away'] else 0), 
        axis=1
    )

    return agg_goals_before_last_day, goals_last_day_sorted


In [5]:
def calculate_points(results, years, win_result):
    points = []
    for result, year in zip(results, years):
        if result == win_result:  # Win condition (1 for home, -1 for away)
            points.append(2 if year <= 1992 else 3)
        elif result == 0:  # Draw condition
            points.append(1)
        else:  # Loss condition
            points.append(0)
    return sum(points)

In [6]:
def aggregate_home_away_points(agg_goals_before_last_day):
    """
    Aggregate goals scored, goals conceded, points, and match count
    for both home and away games based on historical point system.
    Only unique matches (different opponents) are counted.
    Returns two DataFrames: home_games and away_games.
    """

    # Remove duplicates to ensure each match is counted only once per home-away combination
    unique_matches = agg_goals_before_last_day.drop_duplicates(subset=['year', 'stage', 'home_team', 'away_team'])

    # Step 1: Home games aggregation with match count and conditional points based on year
    home_games = agg_goals_before_last_day.groupby(['year', 'stage', 'home_team']).agg(
        goals_scored=('goals_home', 'sum'),
        goals_conceded=('goals_away', 'sum'),
        points_home=('won', lambda x: calculate_points(x, agg_goals_before_last_day.loc[x.index, 'year'], 1)),
        match_count_home=('home_team', 'count')
    ).reset_index()


    # Step 2: Away games aggregation with match count and conditional points based on year
    away_games = agg_goals_before_last_day.groupby(['year', 'stage', 'away_team']).agg(
        goals_scored=('goals_away', 'sum'),
        goals_conceded=('goals_home', 'sum'),
        points_away=('won', lambda x: calculate_points(x, agg_goals_before_last_day.loc[x.index, 'year'], -1)),
        match_count_away=('away_team', 'count')
    ).reset_index()

    return home_games, away_games


## tie-break function 

In [7]:
def tiebreaker_before(row1, row2, agg_data):
    """
    Resolves ties between two rows using head-to-head, goal difference, and goals scored criteria.
    """
    print("\n=== Applying Tiebreaker ===")
    print(f"Row1: {row1}")
    print(f"Row2: {row2}\n")

    # Look for the head-to-head match within the same year and stage
    match = agg_data[
        (agg_data['year'] == row1['year']) & (agg_data['stage'] == row1['stage']) &
        (((agg_data['home_team'] == row1['team']) & (agg_data['away_team'] == row2['team'])) |
            ((agg_data['home_team'] == row2['team']) & (agg_data['away_team'] == row1['team'])))
    ]

    # First criterion: Goal difference
    print(f"Goal difference: {row1['team']} = {row1['goals_difference']}, {row2['team']} = {row2['goals_difference']}")
    if row1['goals_difference'] > row2['goals_difference']:
        print(f"Goal difference result: {row1['team']} wins")
        return row1['team'], 1, 0
    elif row1['goals_difference'] < row2['goals_difference']:
        print(f"Goal difference result: {row2['team']} wins")
        return row2['team'], 0, 1

    # Second criterion: Goals scored
    print(f"Goals scored: {row1['team']} = {row1['goals_scored']}, {row2['team']} = {row2['goals_scored']}")
    if row1['goals_scored'] > row2['goals_scored']:
        print(f"Goals scored result: {row1['team']} wins")
        return row1['team'], 1, 0
    elif row1['goals_scored'] < row2['goals_scored']:
        print(f"Goals scored result: {row2['team']} wins")
        return row2['team'], 0, 1
    
    # Third criterion: Head-to-head result
    if not match.empty:
        match_result = match.iloc[0]
        print("Head-to-head match found:")
        print(match_result)

        if match_result['won'] == 1:  # Home team won
            if match_result['home_team'] == row1['team']:
                print(f"Head-to-head result: {row1['team']} wins")
                return row1['team'], 1, 0  # row1's team wins
            else:
                print(f"Head-to-head result: {row2['team']} wins")
                return row2['team'], 0, 1  # row2's team wins
        elif match_result['won'] == -1:  # Away team won
            if match_result['away_team'] == row1['team']:
                print(f"Head-to-head result: {row1['team']} wins")
                return row1['team'], 1, 0  # row1's team wins
            else:
                print(f"Head-to-head result: {row2['team']} wins")
                return row2['team'], 0, 1  # row2's team wins
    else:
        print("No head-to-head match found.")

    # If all criteria are still tied, mark as a tie
    print("All criteria tied. Result: tie")
    return 'tie', 0, 0


In [8]:
# This function does NOT consider head-to-head results in the last (third) matchday as a tiebreaker
def tiebreaker_after_old(row1, row2, agg_data):
    """
    Resolves ties between two rows using head-to-head, goal difference, and goals scored criteria.
    """
    print("\n=== Applying Tiebreaker ===")
    print(f"Row1: {row1}")
    print(f"Row2: {row2}\n")

    # Look for the head-to-head match within the same year and stage
    match = agg_data[
        (agg_data['year'] == row1['year']) & (agg_data['stage'] == row1['stage']) &
        (((agg_data['home_team'] == row1['team']) & (agg_data['away_team'] == row2['team'])) |
            ((agg_data['home_team'] == row2['team']) & (agg_data['away_team'] == row1['team'])))
    ]

    # First criterion: Goal difference
    print(f"Goal difference: {row1['team']} = {row1['total_goals_difference']}, {row2['team']} = {row2['total_goals_difference']}")
    if row1['total_goals_difference'] > row2['total_goals_difference']:
        print(f"Goal difference result: {row1['team']} wins")
        return row1['team'], 1, 0
    elif row1['total_goals_difference'] < row2['total_goals_difference']:
        print(f"Goal difference result: {row2['team']} wins")
        return row2['team'], 0, 1

    # Second criterion: Goals scored
    print(f"Goals scored: {row1['team']} = {row1['total_goals_scored']}, {row2['team']} = {row2['total_goals_scored']}")
    if row1['total_goals_scored'] > row2['total_goals_scored']:
        print(f"Goals scored result: {row1['team']} wins")
        return row1['team'], 1, 0
    elif row1['total_goals_scored'] < row2['total_goals_scored']:
        print(f"Goals scored result: {row2['team']} wins")
        return row2['team'], 0, 1
    
    # Third criterion: Head-to-head result
    if not match.empty:
        match_result = match.iloc[0]
        print("Head-to-head match found:")
        print(match_result)

        if match_result['won'] == 1:  # Home team won
            if match_result['home_team'] == row1['team']:
                print(f"Head-to-head result: {row1['team']} wins")
                return row1['team'], 1, 0  # row1's team wins
            else:
                print(f"Head-to-head result: {row2['team']} wins")
                return row2['team'], 0, 1  # row2's team wins
        elif match_result['won'] == -1:  # Away team won
            if match_result['away_team'] == row1['team']:
                print(f"Head-to-head result: {row1['team']} wins")
                return row1['team'], 1, 0  # row1's team wins
            else:
                print(f"Head-to-head result: {row2['team']} wins")
                return row2['team'], 0, 1  # row2's team wins
    else:
        print("No head-to-head match found.")

    # If all criteria are still tied, mark as a tie
    print("All criteria tied. Result: tie")
    return 'tie', 0, 0


In [9]:
# This function does consider head-to-head results in the last (third) matchday as a tiebreaker

def tiebreaker_after(row1, row2, agg_data):
    """
    Resolves ties between two rows using head-to-head, goal difference, and goals scored criteria.
    """
    print("\n=== Applying Tiebreaker ===")
    print(f"Row1: {row1}")
    print(f"Row2: {row2}\n")

    # Look for the head-to-head match within the same year and stage
    match = agg_data[
        (agg_data['year'] == row1['year']) & (agg_data['stage'] == row1['stage']) &
        (((agg_data['home_team'] == row1['team']) & (agg_data['away_team'] == row2['team'])) |
            ((agg_data['home_team'] == row2['team']) & (agg_data['away_team'] == row1['team'])))
    ]

    # First criterion: Goal difference
    print(f"Goal difference: {row1['team']} = {row1['total_goals_difference']}, {row2['team']} = {row2['total_goals_difference']}")
    if row1['total_goals_difference'] > row2['total_goals_difference']:
        print(f"Goal difference result: {row1['team']} wins")
        return row1['team'], 1, 0
    elif row1['total_goals_difference'] < row2['total_goals_difference']:
        print(f"Goal difference result: {row2['team']} wins")
        return row2['team'], 0, 1

    # Second criterion: Goals scored
    print(f"Goals scored: {row1['team']} = {row1['total_goals_scored']}, {row2['team']} = {row2['total_goals_scored']}")
    if row1['total_goals_scored'] > row2['total_goals_scored']:
        print(f"Goals scored result: {row1['team']} wins")
        return row1['team'], 1, 0
    elif row1['total_goals_scored'] < row2['total_goals_scored']:
        print(f"Goals scored result: {row2['team']} wins")
        return row2['team'], 0, 1
    
    # Third criterion: Head-to-head result
    if not match.empty:
        match_result = match.iloc[0]
        print("Head-to-head match found:")
        print(match_result)

        if match_result['won'] == 1:  # Home team won
            if match_result['home_team'] == row1['team']:
                print(f"Head-to-head result: {row1['team']} wins")
                return row1['team'], 1, 0  # row1's team wins
            else:
                print(f"Head-to-head result: {row2['team']} wins")
                return row2['team'], 0, 1  # row2's team wins
        elif match_result['won'] == -1:  # Away team won
            if match_result['away_team'] == row1['team']:
                print(f"Head-to-head result: {row1['team']} wins")
                return row1['team'], 1, 0  # row1's team wins
            else:
                print(f"Head-to-head result: {row2['team']} wins")
                return row2['team'], 0, 1  # row2's team wins
    else:
        print("No head-to-head match found.")
        
        # Check last game performance if no head-to-head match
        row1_last_game_performance = row1["last_game_goals_scored"] - row1["last_game_goals_conceded"]
        row2_last_game_performance = row2["last_game_goals_scored"] - row2["last_game_goals_conceded"]
        print(f"Last game performance: {row1['team']} = {row1_last_game_performance}, {row2['team']} = {row2_last_game_performance}")
        
        if row1_last_game_performance > row2_last_game_performance:
            print(f"Last game result: {row1['team']} wins")
            return row1['team'], 1, 0
        elif row1_last_game_performance < row2_last_game_performance:
            print(f"Last game result: {row2['team']} wins")
            return row2['team'], 0, 1

    # If all criteria are still tied, mark as a tie
    print("All criteria tied. Result: tie")
    return 'tie', 0, 0


In [10]:
def fifa_before_last(home_games, away_games, agg_goals_before_last_day, team_counts):
    """
    Process home and away games data to aggregate goals, points, and standings, including handling ties
    and adjusting for unplayed or 0-0 draws.
    """

    # Step 1: Merge home_games and away_games on year, stage, home_team with away_team
    all_games_before_last = pd.merge(
        home_games,
        away_games,
        left_on=['year', 'stage', 'home_team'],
        right_on=['year', 'stage', 'away_team'],
        how='outer',
        suffixes=('_home', '_away')
    )

    # Assign teams and handle missing values
    # Combine home_team and away_team into a single 'team' column and calculate goals, points, and matches
    all_games_before_last['team'] = all_games_before_last['home_team'].fillna(all_games_before_last['away_team'])
    all_games_before_last['goals_scored'] = all_games_before_last['goals_scored_home'].fillna(0) + all_games_before_last['goals_scored_away'].fillna(0)
    all_games_before_last['goals_conceded'] = all_games_before_last['goals_conceded_home'].fillna(0) + all_games_before_last['goals_conceded_away'].fillna(0)
    all_games_before_last['points'] = all_games_before_last['points_home'].fillna(0) + all_games_before_last['points_away'].fillna(0)
    all_games_before_last['total_matches'] = all_games_before_last['match_count_home'].fillna(0) + all_games_before_last['match_count_away'].fillna(0)

    # Ensure all teams from team_counts (using team_list) are included
    team_counts = team_counts.explode('team_list').rename(columns={'team_list': 'team'})

    # Select unique teams and merge
    all_teams = team_counts[['year', 'stage', 'team']].drop_duplicates()
    all_games_before_last = all_teams.merge(
        all_games_before_last,
        on=['year', 'stage', 'team'],
        how='left'
    )

    # Check for observations with missing values in the specified columns
    missing_values = all_games_before_last[
        all_games_before_last[['goals_scored', 'goals_conceded', 'points', 'total_matches']].isnull().any(axis=1)
    ]

    # Print observations with missing values
    if not missing_values.empty:
        print("Observations with missing values before filling:")
        print(missing_values)
    else:
        print("No missing values in the specified columns.")

    # Fill missing values for teams with no activity
    all_games_before_last = all_games_before_last.fillna({'goals_scored': 0, 'goals_conceded': 0, 'points': 0, 'goals_difference': 0, 'total_matches': 0})
    all_games_before_last['goals_difference'] = all_games_before_last['goals_scored'] - all_games_before_last['goals_conceded']

    # Check for observations where total_matches == 1
    matches_one = all_games_before_last[all_games_before_last['total_matches'] == 1]
    if not matches_one.empty:
        print("Observations where total_matches == 1:")
        print(matches_one)
    else:
        print("No observations where total_matches == 1.")

    # Adjust points for teams with one or no matches
    all_games_before_last.loc[all_games_before_last['total_matches'] == 1, 'points'] += 1
    all_games_before_last.loc[all_games_before_last['total_matches'] == 1, 'total_matches'] = 2

    # Check for observations where total_matches == 0
    matches_zero = all_games_before_last[all_games_before_last['total_matches'] == 0]
    if not matches_zero.empty:
        print("Observations where total_matches == 0:")
        print(matches_zero)
    else:
        print("No observations where total_matches == 0.")

    # Adjust points for teams with no matches
    all_games_before_last.loc[all_games_before_last['total_matches'] == 0, 'points'] += 2
    all_games_before_last.loc[all_games_before_last['total_matches'] == 0, 'total_matches'] = 2


    # Initial sorting by points
    all_games_before_last = all_games_before_last.sort_values(
        by=['year', 'stage', 'points'],
        ascending=[True, True, False]
    ).reset_index(drop=True)


    # Apply tie-breaking for each pair of tied teams
    all_games_before_last['tiebreaker'] = 'no need'
    all_games_before_last['tie_won'] = 0

    for i in range(len(all_games_before_last) - 1):
        row1 = all_games_before_last.iloc[i]
        row2 = all_games_before_last.iloc[i + 1]

        # Check if rows are tied in points within the same year and stage
        if row1['year'] == row2['year'] and row1['stage'] == row2['stage'] and row1['points'] == row2['points']:
            tiebreak_result, tie_won_row1, tie_won_row2 = tiebreaker_before(row1, row2, agg_goals_before_last_day)

            if tiebreak_result != 'tie':
                all_games_before_last.at[i, 'tiebreaker'] = tiebreak_result
                all_games_before_last.at[i, 'tie_won'] = tie_won_row1
                all_games_before_last.at[i + 1, 'tiebreaker'] = tiebreak_result
                all_games_before_last.at[i + 1, 'tie_won'] = tie_won_row2

    # Final sorting by all criteria
    all_games_before_last = all_games_before_last.sort_values(
        by=['year', 'stage', 'points', 'goals_difference', 'goals_scored','tie_won'],
        ascending=[True, True, False, False, False, False]
    ).reset_index(drop=True)

    # Assign standings within each group
    all_games_before_last['standing'] = all_games_before_last.groupby(['year', 'stage']).cumcount() + 1

    # Retain only relevant columns
    all_games_before_last = all_games_before_last[
        ['year', 'stage', 'team', 'goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches', 'standing', 'tiebreaker', 'tie_won']
    ]

    # Convert numerical columns to integers
    all_games_before_last[['goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches', 'standing']] = all_games_before_last[
        ['goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches', 'standing']].astype(int)

    return all_games_before_last



# last day league standing and changes

In [11]:
def fifa_final_wc(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day):
    """
    Process and track team performance for the final stage of a given UEFA Euro tournament,
    including handling the last match day's goals and updating standings.
    """

    # Step 1: Filter the data for the specific year and stage
    group_goals_tracking = all_games_before_last[
        (all_games_before_last['year'] == year) & 
        (all_games_before_last['stage'] == stage)
    ].copy()

    group_goals_last_day = goals_last_day_sorted[
        (goals_last_day_sorted['year'] == year) & 
        (goals_last_day_sorted['stage'] == stage)
    ]

    # Step 2: Initialize columns for tracking team performance
    group_goals_tracking['before_last_game_goals_scored'] = group_goals_tracking['goals_scored']
    group_goals_tracking['before_last_game_goals_conceded'] = group_goals_tracking['goals_conceded']
    group_goals_tracking['before_last_game_standing'] = group_goals_tracking['standing']
    group_goals_tracking['before_last_game_points'] = group_goals_tracking['points']


    group_goals_tracking['last_game_goals_scored'] = 0
    group_goals_tracking['last_game_goals_conceded'] = 0
    group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored']
    group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded']
    group_goals_tracking['total_goals_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']
    group_goals_tracking['last_game_points'] = 0
    group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points']

    # Remove unecessary columns
    group_goals_tracking = group_goals_tracking.drop(columns=['goals_difference','goals_scored', 'goals_conceded', 'standing', 'points',
                                                             'total_matches'])

    # Initialize last_game_standing to the initial standings
    group_goals_tracking['last_game_standing'] = group_goals_tracking['before_last_game_standing']

    # Add one point to each team for a 0-0 starting score
    group_goals_tracking['total_points'] += 1

    # Initialize position counters based on initial standings
    group_goals_tracking['1st'] = group_goals_tracking['before_last_game_standing'].apply(lambda x: 1 if x == 1 else 0)
    group_goals_tracking['2nd'] = group_goals_tracking['before_last_game_standing'].apply(lambda x: 1 if x == 2 else 0)
    group_goals_tracking['3rd'] = group_goals_tracking['before_last_game_standing'].apply(lambda x: 1 if x == 3 else 0)
    group_goals_tracking['4th'] = group_goals_tracking['before_last_game_standing'].apply(lambda x: 1 if x == 4 else 0)

    group_goals_tracking['changes'] = 0  # Initialize this but will be redefined later as the sum of 1st, 2nd, 3rd, 4th
    group_goals_tracking['tied'] = False  # Initialize a flag to track tied teams
    group_goals_tracking['tie_won'] = group_goals_tracking['tie_won'] 

    # Step 3: Sort group_goals_last_day by goal_minute in ascending order
    group_goals_last_day = group_goals_last_day.sort_values(by='goal_minute', ascending=True)

    # Print the year, stage, and standings before starting the loop for last match goals
    print(f"\n=== Initial Standings for Year {year}, {stage} Before Last Match Goals ===\n")
    display_columns = ['team', 'total_points', 'total_goals_scored', 'total_goals_conceded', 
                       'total_goals_difference', 'before_last_game_points', 'before_last_game_standing']
    print(group_goals_tracking[display_columns].to_string(index=False))
    print("\n====================================================\n")

    # Step 4: Iterate through the sorted and filtered last match goals and update the goals_tracking table
    previous_standings = group_goals_tracking['last_game_standing'].copy()
    first_iteration = True  # Variable to track the first iteration

    for _, goal in group_goals_last_day.iterrows():
        home_team = goal['home_team']
        away_team = goal['away_team']
        player_team = goal['scorer_nationality']

        # Print goal information for each goal
        print(f"Analyzing goal: {goal['goal_minute']} minute, Player team: {player_team}, Home: {home_team}, Away: {away_team}")

        # Update the goals based on who scored the goal
        if player_team == home_team:
            # Home team scored, update home scored and away conceded
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_conceded'] += 1
        elif player_team == away_team:
            # Away team scored, update away scored and home conceded
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_conceded'] += 1

        # Step 5: Update total_goals_scored, total_goals_conceded, and total_goals_difference
        group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored'] + group_goals_tracking['last_game_goals_scored']
        group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded'] + group_goals_tracking['last_game_goals_conceded']
        group_goals_tracking['total_goals_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']

        # Step 6: Assign points for the last game dynamically after each goal
        for i, row in group_goals_tracking.iterrows():
            if row['last_game_goals_scored'] > row['last_game_goals_conceded']:
                if year <= 1992:
                    group_goals_tracking.loc[i, 'last_game_points'] = 2  # Win before or during 1992
                else:
                    group_goals_tracking.loc[i, 'last_game_points'] = 3  # Win after 1992
            elif row['last_game_goals_scored'] == row['last_game_goals_conceded']:
                group_goals_tracking.loc[i, 'last_game_points'] = 1  # Draw
            else:
                group_goals_tracking.loc[i, 'last_game_points'] = 0  # Loss

        # Step 7: Update total points
        group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points'] + group_goals_tracking['last_game_points']


        # Step 8a: Mark teams that are tied based on total points
        group_goals_tracking['tied'] = group_goals_tracking.duplicated(subset=['total_points'], keep=False)
        print("\n=== Teams with Identical Points (Tied Teams) ===\n")
        print(group_goals_tracking[group_goals_tracking['tied']][['team', 'total_points']])

        # Reset `tie_won` to 0 for all teams
        group_goals_tracking['tie_won'] = 0

        # Step 8b: Process ties only if tied teams exist
        tied_teams = group_goals_tracking[group_goals_tracking['tied']]

        if not tied_teams.empty:
            print("\n=== Evaluating Head-to-Head for Tied Teams ===\n")
            
            # Iterate through all pairs of tied teams
            for i, row1 in tied_teams.iterrows():
                for j, row2 in tied_teams.iterrows():
                    if i < j:  # Compare each pair only once
                        print(f"Checking tie between: {row1['team']} and {row2['team']}")

                        # Apply tiebreaker function
                        tiebreak_result, tie_won_row1, tie_won_row2 = tiebreaker_after(row1, row2, agg_goals_before_last_day)

                        if tiebreak_result != 'tie':
                            print(f"Tiebreak Result: Winner is {tiebreak_result}")
                        else:
                            print(f"No winner in tiebreak between {row1['team']} and {row2['team']}")

                        # Update the `tie_won` column based on the results
                        group_goals_tracking.loc[group_goals_tracking['team'] == row1['team'], 'tie_won'] += tie_won_row1
                        group_goals_tracking.loc[group_goals_tracking['team'] == row2['team'], 'tie_won'] += tie_won_row2

        # Step 8c: Sort teams by tie-breaking criteria
        group_goals_tracking = group_goals_tracking.sort_values(
            by=['total_points', 'total_goals_difference', 'total_goals_scored', 'tie_won'],
            ascending=[False, False, False, False]
)

        # Step 9: Update standings
        group_goals_tracking['last_game_standing'] = group_goals_tracking.reset_index(drop=True).index + 1
        print("\n=== Updated Standings After Sorting and Tie Break ===\n")
        print(group_goals_tracking[['team', 'last_game_standing', 'total_points', 'total_goals_difference', 'tie_won']])

        # Step 10: Track changes and update standing positions after each goal is processed
        for i, row in group_goals_tracking.iterrows():
            team = row['team']
            if first_iteration:
                if row['before_last_game_standing'] == row['last_game_standing']:
                    continue
                else:
                    if row['last_game_standing'] == 1:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '1st'] += 1
                    elif row['last_game_standing'] == 2:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '2nd'] += 1
                    elif row['last_game_standing'] == 3:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '3rd'] += 1
                    elif row['last_game_standing'] == 4:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '4th'] += 1
                first_iteration = False
            else:
                if row['last_game_standing'] != previous_standings[i]:  
                    if row['last_game_standing'] == 1:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '1st'] += 1
                    elif row['last_game_standing'] == 2:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '2nd'] += 1
                    elif row['last_game_standing'] == 3:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '3rd'] += 1
                    elif row['last_game_standing'] == 4:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '4th'] += 1

        # Update previous standings after each goal
        previous_standings = group_goals_tracking['last_game_standing'].copy()

        # Step 11: Calculate changes as the sum of 1st, 2nd, 3rd, and 4th
        group_goals_tracking['changes'] = group_goals_tracking[['1st', '2nd', '3rd', '4th']].sum(axis=1)

        # Step 12: Print the updated group_goals_tracking after processing each goal
        print("\n=== Updated Standings After This Goal ===\n")
        display_columns = ['team', 'total_points', 'total_goals_scored', 'total_goals_conceded', 
                           'total_goals_difference', 'last_game_points', 'last_game_standing', 
                           'changes', '1st', '2nd', '3rd', '4th', 'tied', 'tie_won']
        print(group_goals_tracking[display_columns].to_string(index=False))
        print("\n========================================\n")

    # Step 13: Return the final DataFrame
    return group_goals_tracking


In [12]:
def track_composition_changes(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day):
    # Step 1: Filter the data for the specific year and stage
    group_goals_tracking = all_games_before_last[
        (all_games_before_last['year'] == year) & 
        (all_games_before_last['stage'] == stage)
    ].copy()

    group_goals_last_day = goals_last_day_sorted[
        (goals_last_day_sorted['year'] == year) & 
        (goals_last_day_sorted['stage'] == stage)
    ]

    # Initialize columns for team performance and standings
    group_goals_tracking['before_last_game_goals_scored'] = group_goals_tracking['goals_scored']
    group_goals_tracking['before_last_game_goals_conceded'] = group_goals_tracking['goals_conceded']
    group_goals_tracking['before_last_game_standing'] = group_goals_tracking['standing']
    group_goals_tracking['before_last_game_points'] = group_goals_tracking['points']
    group_goals_tracking['last_game_goals_scored'] = 0
    group_goals_tracking['last_game_goals_conceded'] = 0
    group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored']
    group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded']
    group_goals_tracking['total_goals_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']
    group_goals_tracking['last_game_points'] = 0
    group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points']
    group_goals_tracking['last_game_standing'] = 0
    group_goals_tracking['tie_won'] = 0  # Initialize tie_won for tiebreak resolution

    # Add one point to each team for a 0-0 starting score
    group_goals_tracking['total_points'] += 1

    # Remove unecessary columns
    group_goals_tracking = group_goals_tracking.drop(columns=['goals_difference','goals_scored', 'goals_conceded', 'standing', 'points',
                                                             'total_matches'])


    # Define top standings limit based on the year (3 for 1992 and earlier, 2 for 1994 and later) for World Cup  *************---------------------------------CHANGE----------------------------------------------*******************
    top_standings_limit = 3 if year <= 1994 else 2 

    # Define top standings limit based on the year (3 for 2016 and later, 2 for 2014 and earlier) for Euros
    # top_standings_limit = 3 if year >= 2016 else 2

    # Print initial standings
    print(f"\n=== Initial Standings for {stage}, {year} (Goal Time = 0) ===")
    print(group_goals_tracking[['team', 'total_points', 'total_goals_scored', 
                                'total_goals_conceded', 'total_goals_difference', 'before_last_game_standing']].to_string(index=False))
    print("\n========================================\n")

    # Step 2: Initialize composition tracking with initial composition (change_num = 0)
    initial_top_teams = set(
        group_goals_tracking[group_goals_tracking['before_last_game_standing'] <= top_standings_limit]['team']
    )

    composition_changes = [{
        'year': year,
        'stage': stage,
        'change_num': 0,
        'goal_time': 0,
        'home_team': None,
        'away_team': None,
        'scorer_team': None,
        'new_top_teams': list(initial_top_teams),
        '1st': group_goals_tracking.loc[group_goals_tracking['before_last_game_standing'] == 1, 'team'].values[0] 
                if (group_goals_tracking['before_last_game_standing'] == 1).any() else None,
        '2nd': group_goals_tracking.loc[group_goals_tracking['before_last_game_standing'] == 2, 'team'].values[0] 
                if (group_goals_tracking['before_last_game_standing'] == 2).any() else None,
        '3rd': group_goals_tracking.loc[group_goals_tracking['before_last_game_standing'] == 3, 'team'].values[0] 
                if (group_goals_tracking['before_last_game_standing'] == 3).any() else None,
        'changed': 0
    }]

    change_counter = 0  # Counter for the number of composition changes

    # Step 3: Sort goals by regulation time
    group_goals_last_day = group_goals_last_day.sort_values(by=['goal_minute'])

    # Step 4: Iterate through each goal and track changes in composition
    for _, goal in group_goals_last_day.iterrows():
        home_team = goal['home_team']
        away_team = goal['away_team']
        scorer_team = goal['scorer_nationality']

        # Update scores based on who scored the goal
        if scorer_team == home_team:
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'total_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_conceded'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'total_goals_conceded'] += 1
        elif scorer_team == away_team:
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'total_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_conceded'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'total_goals_conceded'] += 1

        # Update goal difference
        group_goals_tracking['total_goals_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']

        # Step 5: Update last_game_points based on the current game state
        group_goals_tracking['last_game_points'] = group_goals_tracking.apply(
            lambda row: (2 if year <= 1992 else 3) if row['last_game_goals_scored'] > row['last_game_goals_conceded'] 
            else (1 if row['last_game_goals_scored'] == row['last_game_goals_conceded'] else 0), 
            axis=1
        )

        # Calculate total points by adding last game points to before_last_game_points
        group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points'] + group_goals_tracking['last_game_points']

        # Step 5b: Evaluate ties based on total points
        group_goals_tracking['tied'] = group_goals_tracking.duplicated(subset=['total_points'], keep=False)

        # Select the tied teams
        tied_teams = group_goals_tracking[group_goals_tracking['tied']]

        # Reset `tie_won` column to 0 for all teams
        group_goals_tracking['tie_won'] = 0

        if not tied_teams.empty:
            print("\n=== Evaluating Head-to-Head for Tied Teams ===\n")

            # Iterate through all pairs of tied teams
            for i, row1 in tied_teams.iterrows():
                for j, row2 in tied_teams.iterrows():
                    if i < j:  # Compare each pair only once
                        team1 = row1['team']
                        team2 = row2['team']

                        print(f"Checking tie between: {team1} and {team2}")

                        # Apply tiebreaker function
                        tiebreak_result, tie_won_row1, tie_won_row2 = tiebreaker_after(row1, row2, agg_goals_before_last_day)

                        if tiebreak_result != 'tie':
                            print(f"Tiebreak Result: Winner is {tiebreak_result}")
                        else:
                            print(f"No winner in tiebreak between {team1} and {team2}")

                        # Update the `tie_won` column based on the results
                        group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tie_won'] += tie_won_row1
                        group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tie_won'] += tie_won_row2

        # Sort teams by updated points and tie-breaking criteria
        group_goals_tracking = group_goals_tracking.sort_values(
            by=['total_points', 'total_goals_difference', 'total_goals_scored', 'tie_won'],
            ascending=[False, False, False, False]
        )
        
        # Update standings after sorting
        group_goals_tracking['last_game_standing'] = group_goals_tracking.reset_index(drop=True).index + 1

        # Print standings after each goal
        print(f"\n=== Standings after goal at minute {goal['goal_minute']} in {stage}, edition {year} ===")
        print(group_goals_tracking[['team', 'total_points', 'total_goals_scored', 'total_goals_conceded', 'total_goals_difference', 'before_last_game_standing']].to_string(index=False))
        print("\n========================================\n")

        # Track top teams and composition changes
        current_top_teams = set(group_goals_tracking.nsmallest(top_standings_limit, 'last_game_standing')['team'])
        changed = int(current_top_teams != initial_top_teams)

        if changed:
            change_counter += 1
            initial_top_teams = current_top_teams

        composition_changes.append({
            'year': year,
            'stage': stage,
            'change_num': change_counter,
            'goal_time': goal['goal_minute'],
            'home_team': home_team,
            'away_team': away_team,
            'scorer_team': scorer_team,
            'new_top_teams': list(current_top_teams),
            '1st': group_goals_tracking.iloc[0]['team'],
            '2nd': group_goals_tracking.iloc[1]['team'] if len(group_goals_tracking) > 1 else None,
            '3rd': group_goals_tracking.iloc[2]['team'] if len(group_goals_tracking) > 2 else None,
            'changed': changed,
        })

    return pd.DataFrame(composition_changes)


# gap between qualifying and not qualifying 

In [13]:
def gap_composition(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day):
    # Step 1: Filter the data for the specific year and stage
    group_goals_tracking = all_games_before_last[
        (all_games_before_last['year'] == year) & 
        (all_games_before_last['stage'] == stage)
    ].copy()

    group_goals_last_day = goals_last_day_sorted[
        (goals_last_day_sorted['year'] == year) & 
        (goals_last_day_sorted['stage'] == stage)
    ]

    # Initialize columns for team performance and standings
    group_goals_tracking['before_last_game_goals_scored'] = group_goals_tracking['goals_scored']
    group_goals_tracking['before_last_game_goals_conceded'] = group_goals_tracking['goals_conceded']
    group_goals_tracking['before_last_game_standing'] = group_goals_tracking['standing']
    group_goals_tracking['before_last_game_points'] = group_goals_tracking['points']
    group_goals_tracking['last_game_goals_scored'] = 0
    group_goals_tracking['last_game_goals_conceded'] = 0
    group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored']
    group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded']
    group_goals_tracking['total_goals_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']
    group_goals_tracking['last_game_points'] = 0
    group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points']
    group_goals_tracking['last_game_standing'] = 0
    group_goals_tracking['tie_won'] = 0  # Initialize tie_won for tiebreak resolution

    # Add one point to each team for a 0-0 starting score
    group_goals_tracking['total_points'] += 1

    # Remove unecessary columns
    group_goals_tracking = group_goals_tracking.drop(columns=['goals_difference','goals_scored', 'goals_conceded', 'standing', 'points',
                                                             'total_matches', 'tiebreaker'])


    # Define top standings limit based on the year (3 for 1992 and earlier, 2 for 1994 and later) for World Cup  *************---------------------------------CHANGE----------------------------------------------*******************
    top_standings_limit = 3 if year <= 1994 else 2 

    # Define top standings limit based on the year (3 for 2016 and later, 2 for 2014 and earlier) for Euros
    # top_standings_limit = 3 if year >= 2016 else 2

    # Print initial standings
    print(f"\n=== STEP 1: Initial Standings for {stage}, {year} (Goal Time = 0) ===")
    print(group_goals_tracking[['team', 'total_points', 'total_goals_scored', 
                                'total_goals_conceded', 'total_goals_difference', 'before_last_game_standing']].to_string(index=False))
    print("\n========================================\n")

    # Initialize composition tracking with initial composition (change_num = 0)
    initial_top_teams = set(
        group_goals_tracking[group_goals_tracking['before_last_game_standing'] <= top_standings_limit]['team']
    )

    composition_changes = [{
        'year': year,
        'stage': stage,
        'change_num': 0,
        'goal_time': 0,
        'home_team': None,
        'away_team': None,
        'scorer_team': None,
        'new_top_teams': list(initial_top_teams),
        '1st': group_goals_tracking.loc[group_goals_tracking['before_last_game_standing'] == 1, 'team'].values[0] 
                if (group_goals_tracking['before_last_game_standing'] == 1).any() else None,
        '2nd': group_goals_tracking.loc[group_goals_tracking['before_last_game_standing'] == 2, 'team'].values[0] 
                if (group_goals_tracking['before_last_game_standing'] == 2).any() else None,
        '3rd': group_goals_tracking.loc[group_goals_tracking['before_last_game_standing'] == 3, 'team'].values[0] 
                if (group_goals_tracking['before_last_game_standing'] == 3).any() else None,
        'changed': 0,
        'points_diff': 0,
        'goals_diff': 0

    }]

    change_counter = 0  # Counter for the number of composition changes

    # Sort goals by regulation time
    group_goals_last_day = group_goals_last_day.sort_values(by=['goal_minute'])

    # Iterate through each goal and track changes in composition
    for _, goal in group_goals_last_day.iterrows():
        home_team = goal['home_team']
        away_team = goal['away_team']
        scorer_team = goal['scorer_nationality']

        # Update scores based on who scored the goal
        if scorer_team == home_team:
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'total_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_conceded'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'total_goals_conceded'] += 1
        elif scorer_team == away_team:
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'total_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_conceded'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'total_goals_conceded'] += 1

        # Update goal difference
        group_goals_tracking['total_goals_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']

        # Update last_game_points based on the current game state
        group_goals_tracking['last_game_points'] = group_goals_tracking.apply(
            lambda row: (2 if year <= 1992 else 3) if row['last_game_goals_scored'] > row['last_game_goals_conceded'] 
            else (1 if row['last_game_goals_scored'] == row['last_game_goals_conceded'] else 0), 
            axis=1
        )

        # Calculate total points by adding last game points to before_last_game_points
        group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points'] + group_goals_tracking['last_game_points']

        # Evaluate ties based on total points
        group_goals_tracking['tied'] = group_goals_tracking.groupby('total_points')['team'].transform('size') > 1

        # Select the tied teams
        tied_teams = group_goals_tracking[group_goals_tracking['tied']]

        # Count the number of tied teams
        num_tied_teams = tied_teams['team'].nunique()

        print(f"\n=== Tied after goal at minute {goal['goal_minute']} by {goal['scorer_nationality']} in {stage}, edition {year} ===")
        print(f"Number of tied teams: {num_tied_teams}")
        print(tied_teams[['team', 'total_points', 'total_goals_scored', 'total_goals_conceded', 'total_goals_difference']])

        # Reset `tie_won` column to 0 for all teams
        group_goals_tracking['tie_won'] = 0

        if not tied_teams.empty:
            print("\n=== STEP 2: Evaluating Head-to-Head for Tied Teams ===\n")

            # Group tied teams by total_points and process each group separately
            for points, tied_group in tied_teams.groupby('total_points'):
                if len(tied_group) > 1:  # Process groups with more than one team
                    print(f"\n=== Checking ties for teams with {points} points ===")
                    
                    # Sort tied_group by additional criteria to ensure proper order
                    tied_group = tied_group.sort_values(by=['total_goals_difference', 'total_goals_scored'], ascending=False)
                    
                    # Iterate through all pairs of tied teams in the current group
                    for i, row1 in tied_group.iterrows():
                        for j, row2 in tied_group.iterrows():
                            if i < j:  # Compare each pair only once
                                team1 = row1['team']
                                team2 = row2['team']

                                print(f"Checking tie between: {team1} and {team2}")

                                # Apply tiebreaker function
                                tiebreak_result, tie_won_row1, tie_won_row2 = tiebreaker_after(row1, row2, agg_goals_before_last_day)

                                if tiebreak_result != 'tie':
                                    print(f"Tiebreak Result: Winner is {tiebreak_result}")
                                else:
                                    print(f"No winner in tiebreak between {team1} and {team2}")

                                # Update the `tie_won` column based on the results
                                group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tie_won'] += tie_won_row1
                                group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tie_won'] += tie_won_row2


        # Sort teams by updated points and tie-breaking criteria
        group_goals_tracking = group_goals_tracking.sort_values(
            by=['total_points', 'total_goals_difference', 'total_goals_scored', 'tie_won'],
            ascending=[False, False, False, False]
        )
        
        # Update standings after sorting
        group_goals_tracking['last_game_standing'] = group_goals_tracking.reset_index(drop=True).index + 1

        # Print standings after each goal
        print(f"\n=== STEP 3: Standings after goal at minute {goal['goal_minute']} by {goal['scorer_nationality']} in {stage}, edition {year} ===")
        print(group_goals_tracking[['team', 'total_points', 'total_goals_scored', 'total_goals_conceded', 'total_goals_difference', 'before_last_game_standing']].to_string(index=False))
        print("\n========================================\n")

        # Initialize tiebreak_result and other variables before the conditions
        tiebreak_result = None
        original_goals_scored_row2 = None
        original_goals_difference_row2 = None
        tie_won_row1 = 0
        tie_won_row2 = 0


        # Calculate points_diff and goals_diff, then apply the tiebreaker if conditions are met
        if top_standings_limit == 3:
            # Calculate points_diff for top_standings_limit == 3
            points_diff = group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 3, 'total_points'].values[0] - \
                        group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 4, 'total_points'].values[0]

            # Calculate goals_diff for top_standings_limit == 3
            goals_diff = group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 3, 'total_goals_difference'].values[0] - \
                        group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 4, 'total_goals_difference'].values[0]
            
            # Add points_last variable to check if the fourth team is drawing its match 
            points_last = group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 4, 'last_game_points'].values[0]
            

           # Apply tiebreaker for three-points-win system after 2012
            if (points_diff == 2 and goals_diff <= 1 and points_last == 1):
                print("\n=== STEP 4: Applying Potential Tiebreaker for 3nd and 4th Place ===\n")
                team_3rd = group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 3, 'team'].values[0]
                team_4th = group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 4, 'team'].values[0]

                # Get rows for the 3rd and 4th teams
                row1 = group_goals_tracking[group_goals_tracking['team'] == team_3rd].iloc[0]
                row2_index = group_goals_tracking[group_goals_tracking['team'] == team_4th].index[0]  # Get index for row2

                # Save original values for resetting later
                original_goals_scored_row2 = group_goals_tracking.loc[row2_index, 'total_goals_scored']
                original_goals_difference_row2 = group_goals_tracking.loc[row2_index, 'total_goals_difference']

                # Increment total_goals_scored and total_goals_difference for row2
                group_goals_tracking.loc[row2_index, 'total_goals_scored'] += 1
                group_goals_tracking.loc[row2_index, 'total_goals_difference'] += 1

                # Retrieve updated row2 after incrementing values
                row2 = group_goals_tracking.loc[row2_index]

                # Apply tiebreaker
                tiebreak_result, tie_won_row1, tie_won_row2 = tiebreaker_after(row1, row2, agg_goals_before_last_day)

                # Reset total_goals_scored and total_goals_difference to their original values
                group_goals_tracking.loc[row2_index, 'total_goals_scored'] = original_goals_scored_row2
                group_goals_tracking.loc[row2_index, 'total_goals_difference'] = original_goals_difference_row2

        elif top_standings_limit == 2:
            # Calculate points_diff for top_standings_limit == 2
            points_diff = group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 2, 'total_points'].values[0] - \
                        group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 3, 'total_points'].values[0]

            # Calculate goals_diff for top_standings_limit == 2
            goals_diff = group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 2, 'total_goals_difference'].values[0] - \
                        group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 3, 'total_goals_difference'].values[0]
            
            # Add points_last variable to check if the fourth team is drawing its match 
            points_last = group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 3, 'last_game_points'].values[0]
            
            # Apply tiebreaker for two-points-win based system before 1992 OR three-points-win system after 1992
            if (points_diff == 1 and goals_diff <= 1 and points_last == 1 and year <= 1992) or \
    (points_diff == 2 and goals_diff <= 1 and points_last == 1 and year > 1992):
                print("\n=== STEP 4: Applying Potential Tiebreaker for 2nd and 3rd Place ===\n")
                team_2nd = group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 2, 'team'].values[0]
                team_3rd = group_goals_tracking.loc[group_goals_tracking['last_game_standing'] == 3, 'team'].values[0]

                # Get rows for the 2nd and 3rd teams
                row1 = group_goals_tracking[group_goals_tracking['team'] == team_2nd].iloc[0]
                row2_index = group_goals_tracking[group_goals_tracking['team'] == team_3rd].index[0]  # Get index for row2

                # Save original values for resetting later
                original_goals_scored_row2 = group_goals_tracking.loc[row2_index, 'total_goals_scored']
                original_goals_difference_row2 = group_goals_tracking.loc[row2_index, 'total_goals_difference']

                # Increment total_goals_scored and total_goals_difference for row2
                group_goals_tracking.loc[row2_index, 'total_goals_scored'] += 1
                group_goals_tracking.loc[row2_index, 'total_goals_difference'] += 1

                # Retrieve updated row2 after incrementing values
                row2 = group_goals_tracking.loc[row2_index]

                # Apply tiebreaker
                tiebreak_result, tie_won_row1, tie_won_row2 = tiebreaker_after (row1, row2, agg_goals_before_last_day)

                # Reset total_goals_scored and total_goals_difference to their original values
                group_goals_tracking.loc[row2_index, 'total_goals_scored'] = original_goals_scored_row2
                group_goals_tracking.loc[row2_index, 'total_goals_difference'] = original_goals_difference_row2

        # Track top teams and composition changes
        current_top_teams = set(group_goals_tracking.nsmallest(top_standings_limit, 'last_game_standing')['team'])
        changed = int(current_top_teams != initial_top_teams)

        if changed:
            change_counter += 1
            initial_top_teams = current_top_teams

        composition_changes.append({
            'year': year,
            'stage': stage,
            'date': goal['short_date'],
            'time':goal['local_time'],
            'change_num': change_counter,
            'goal_time': goal['goal_minute'],
            'home_team': home_team,
            'away_team': away_team,
            'scorer_team': scorer_team,
            'new_top_teams': list(current_top_teams),
            '1st': group_goals_tracking.iloc[0]['team'],
            '2nd': group_goals_tracking.iloc[1]['team'] if len(group_goals_tracking) > 1 else None,
            '3rd': group_goals_tracking.iloc[2]['team'] if len(group_goals_tracking) > 2 else None,
            'changed': changed,
            'points_diff': points_diff,
            'goals_diff': goals_diff,
            'tiebreak_result': tiebreak_result,
        })

    return pd.DataFrame(composition_changes)


# best four third_placed

## European Championship

### men

In [14]:
def best_four_third_placed_eu_men(goals_last_day_sorted, all_games_before_last, agg_goals_before_last_day):
    """
    Process goals and calculate standings for third-placed teams, returning stats after each goal,
    with a single column per group containing all third-team info. Adds a list of teams excluded from
    the best 4 third-placed teams after evaluating each goal and a changes variable that increments
    every time the composition of top4_third_teams changes.
    """
    # Filter data for years >= 2016
    all_games_before_last = all_games_before_last[(all_games_before_last['year'] >= 2016)].copy()
    all_games_before_last['year'] = all_games_before_last['year'].astype(int)

    # Add 1 point to each team for the assumed 0-0 starting score
    all_games_before_last['points'] += 1

    results = []

    # Group by year and process each year separately
    for year, year_data in all_games_before_last.groupby('year'):
        year = int(year)
        print(f"\n--- Processing Year: {year} ---")
        year_data = year_data.copy()

        # Initialize columns for tracking stats
        year_data['before_last_game_goals_scored'] = year_data['goals_scored']
        year_data['before_last_game_goals_conceded'] = year_data['goals_conceded']
        year_data['before_last_game_points'] = year_data['points']
        year_data['last_game_goals_scored'] = 0
        year_data['last_game_goals_conceded'] = 0
        year_data['total_goals_scored'] = year_data['before_last_game_goals_scored']
        year_data['total_goals_conceded'] = year_data['before_last_game_goals_conceded']
        year_data['total_goals_difference'] = year_data['total_goals_scored'] - year_data['total_goals_conceded']
        year_data['last_game_points'] = 0
        year_data['total_points'] = year_data['before_last_game_points']

        # Sort goals by time for the current year
        goals_last_day_year = goals_last_day_sorted[(goals_last_day_sorted['year'] == year)]

        # Initialize a set to track locked stages
        locked_stages = set()

        # Initial standings for goal_time = 0
        third_teams_info = {}
        year_data['tied_won'] = 0  # Initialize tied_won column

        # Sort teams by the `standing` variable
        sorted_standings = year_data.sort_values(by='standing', ascending=True)

        # Extract third-placed teams based on sorted order
        for group_stage, group_data in sorted_standings.groupby('stage'):
            group_key = group_stage.replace(" ", "_")  # Ensure valid keys
            if len(group_data) >= 3:
                third_placed = group_data.iloc[2]  # Third-placed team based on `standing`
                third_teams_info[f'third_team_info_{group_key}'] = {
                    'team': third_placed['team'],
                    'points': int(third_placed['total_points']),
                    'goals_difference': int(third_placed['total_goals_difference']),
                    'goals_scored': int(third_placed['total_goals_scored'])
                }
            else:
                third_teams_info[f'third_team_info_{group_key}'] = None

        # Determine the top 4 third-placed teams initially
        sorted_third_teams = sorted(
            [(info['team'], info['points'], info['goals_difference'], info['goals_scored']) for info in third_teams_info.values() if info],
            key=lambda x: (x[1], x[2], x[3]),
            reverse=True
        )
        top4_third_teams = [team[0] for team in sorted_third_teams[:4]]
        excluded_teams = [team[0] for team in sorted_third_teams[4:]]
        previous_top4 = top4_third_teams
        changes = 0

        # Add the initial state row for the year
        results.append({
            'year': year,
            'stage': None,
            'goal_time': 0,
            'home_team': None,
            'away_team': None,
            'scorer_team': None,
            **third_teams_info,
            'top4_third_teams': top4_third_teams,
            'excluded_teams': excluded_teams,
            'changes': changes
        })

        # Process goals
        for _, goal in goals_last_day_year.iterrows():
            stage = goal['stage']
            home_team = goal['home_team']
            away_team = goal['away_team']
            player_team = goal['scorer_nationality']

            # Lock stats for stages before the current one
            if stage not in locked_stages:
                for locked_stage in year_data['stage'].unique():
                    if locked_stage != stage and locked_stage not in locked_stages:
                        locked_stages.add(locked_stage)

            # Update scores for the current stage
            if stage not in locked_stages:
                opponent_team = home_team if player_team == away_team else away_team
                year_data.loc[year_data['team'] == player_team, 'last_game_goals_scored'] += 1
                year_data.loc[year_data['team'] == opponent_team, 'last_game_goals_conceded'] += 1

                # Update total goals scored, goals conceded, and goal difference
                year_data['total_goals_scored'] = year_data['before_last_game_goals_scored'] + year_data['last_game_goals_scored']
                year_data['total_goals_conceded'] = year_data['before_last_game_goals_conceded'] + year_data['last_game_goals_conceded']
                year_data['total_goals_difference'] = year_data['total_goals_scored'] - year_data['total_goals_conceded']

                # Adjust points dynamically for each game outcome
                year_data['last_game_points'] = (
                    (year_data['last_game_goals_scored'] > year_data['last_game_goals_conceded']).astype(int) * 2 +
                    (year_data['last_game_goals_scored'] == year_data['last_game_goals_conceded']).astype(int) * 0 +
                    (year_data['last_game_goals_scored'] < year_data['last_game_goals_conceded']).astype(int) * -1
                )

                # Update total points
                year_data['total_points'] = year_data['before_last_game_points'] + year_data['last_game_points']

            # Recalculate third-placed team info for each group
            third_teams_info = {}

            for group_stage, group_data in year_data.groupby('stage'):
                group_key = group_stage.replace(" ", "_")  # Ensure valid keys

                # Sort teams by total points, goal difference, and goals scored
                sorted_standings = group_data.sort_values(
                    by=['total_points', 'total_goals_difference', 'total_goals_scored'], 
                    ascending=[False, False, False]
                )

                # Check for ties in total points
                tied_teams = sorted_standings[sorted_standings.duplicated(subset=['total_points'], keep=False)]

                if not tied_teams.empty:
                    # Resolve ties using tiebreaker_after
                    for i, row1 in tied_teams.iterrows():
                        for j, row2 in tied_teams.iterrows():
                            if i != j:  # Ensure no self-comparison
                                winner, _, _ = tiebreaker_after(row1, row2, agg_goals_before_last_day)
                                if winner == row1['team']:
                                    sorted_standings.loc[sorted_standings['team'] == row1['team'], 'tied_won'] += 1

                # Final sorting with tie-breaking criteria
                sorted_standings = sorted_standings.sort_values(
                    by=['total_points', 'tied_won', 'total_goals_difference', 'total_goals_scored'], 
                    ascending=[False, False, False, False]
                )

                # Extract third-placed team
                if len(sorted_standings) >= 3:
                    third_placed = sorted_standings.iloc[2]
                    third_teams_info[f'third_team_info_{group_key}'] = {
                        'team': third_placed['team'],
                        'points': int(third_placed['total_points']),
                        'goals_difference': int(third_placed['total_goals_difference']),
                        'goals_scored': int(third_placed['total_goals_scored'])
                    }
                else:
                    third_teams_info[f'third_team_info_{group_key}'] = None

            # Sort third-placed teams across groups and get top 4
            sorted_third_teams = sorted(
                [(info['team'], info['points'], info['goals_difference'], info['goals_scored']) for info in third_teams_info.values() if info],
                key=lambda x: (x[1], x[2], x[3]),
                reverse=True
            )
            top4_third_teams = [team[0] for team in sorted_third_teams[:4]]
            excluded_teams = [team[0] for team in sorted_third_teams[4:]]

            # Check for changes in top4_third_teams
            if top4_third_teams != previous_top4:
                changes += 1
                previous_top4 = top4_third_teams

            results.append({
                'year': year,
                'stage': stage,
                'goal_time': goal['goal_minute'],
                'home_team': home_team,
                'away_team': away_team,
                'scorer_team': player_team,
                **third_teams_info,
                'top4_third_teams': top4_third_teams,
                'excluded_teams': excluded_teams,
                'changes': changes
            })

    return pd.DataFrame(results)

### women

In [15]:
def best_two_third_placed_eu_women(goals_last_day_sorted, all_games_before_last, agg_goals_before_last_day):
    """
    Process goals and calculate standings for third-placed teams, returning stats after each goal,
    with a single column per group containing all third-team info. Adds a list of teams excluded from
    the best 2 third-placed teams after evaluating each goal and a changes variable that increments
    every time the composition of top2_third_teams changes.
    """
    # Apply filter to process only years 2009 and 2013
    all_games_before_last = all_games_before_last[(all_games_before_last['year'].isin([2009, 2013]))].copy()    
    all_games_before_last['year'] = all_games_before_last['year'].astype(int)

    # Add 1 point to each team for the assumed 0-0 starting score
    all_games_before_last['points'] += 1

    results = []

    # Group by year and process each year separately
    for year, year_data in all_games_before_last.groupby('year'):
        year = int(year)
        print(f"\n--- Processing Year: {year} ---")
        year_data = year_data.copy()

        # Initialize columns for tracking stats
        year_data['before_last_game_goals_scored'] = year_data['goals_scored']
        year_data['before_last_game_goals_conceded'] = year_data['goals_conceded']
        year_data['before_last_game_points'] = year_data['points']
        year_data['last_game_goals_scored'] = 0
        year_data['last_game_goals_conceded'] = 0
        year_data['total_goals_scored'] = year_data['before_last_game_goals_scored']
        year_data['total_goals_conceded'] = year_data['before_last_game_goals_conceded']
        year_data['total_goals_difference'] = year_data['total_goals_scored'] - year_data['total_goals_conceded']
        year_data['last_game_points'] = 0
        year_data['total_points'] = year_data['before_last_game_points']

        # Sort goals by time for the current year
        goals_last_day_year = goals_last_day_sorted[(goals_last_day_sorted['year'] == year)]

        # Initialize a set to track locked stages
        locked_stages = set()

        # Initial standings for goal_time = 0
        third_teams_info = {}
        year_data['tied_won'] = 0  # Initialize tied_won column

        # Sort teams by the `standing` variable
        sorted_standings = year_data.sort_values(by='standing', ascending=True)

        # Extract third-placed teams based on sorted order
        for group_stage, group_data in sorted_standings.groupby('stage'):
            group_key = group_stage.replace(" ", "_")  # Ensure valid keys
            if len(group_data) >= 3:
                third_placed = group_data.iloc[2]  # Third-placed team based on `standing`
                third_teams_info[f'third_team_info_{group_key}'] = {
                    'team': third_placed['team'],
                    'points': int(third_placed['total_points']),
                    'goals_difference': int(third_placed['total_goals_difference']),
                    'goals_scored': int(third_placed['total_goals_scored'])
                }
            else:
                third_teams_info[f'third_team_info_{group_key}'] = None

        # Determine the top 2 third-placed teams initially
        sorted_third_teams = sorted(
            [(info['team'], info['points'], info['goals_difference'], info['goals_scored']) for info in third_teams_info.values() if info],
            key=lambda x: (x[1], x[2], x[3]),
            reverse=True
        )
        top2_third_teams = [team[0] for team in sorted_third_teams[:2]]
        excluded_teams = [team[0] for team in sorted_third_teams[2:]]
        previous_top2 = top2_third_teams
        changes = 0

        # Add the initial state row for the year
        results.append({
            'year': year,
            'stage': None,
            'goal_time': 0,
            'home_team': None,
            'away_team': None,
            'scorer_team': None,
            **third_teams_info,
            'top2_third_teams': top2_third_teams,
            'excluded_teams': excluded_teams,
            'changes': changes
        })

        # Process goals
        for _, goal in goals_last_day_year.iterrows():
            stage = goal['stage']
            home_team = goal['home_team']
            away_team = goal['away_team']
            player_team = goal['scorer_nationality']

            # Lock stats for stages before the current one
            if stage not in locked_stages:
                for locked_stage in year_data['stage'].unique():
                    if locked_stage != stage and locked_stage not in locked_stages:
                        locked_stages.add(locked_stage)

            # Update scores for the current stage
            if stage not in locked_stages:
                opponent_team = home_team if player_team == away_team else away_team
                year_data.loc[year_data['team'] == player_team, 'last_game_goals_scored'] += 1
                year_data.loc[year_data['team'] == opponent_team, 'last_game_goals_conceded'] += 1

                # Update total goals scored, goals conceded, and goal difference
                year_data['total_goals_scored'] = year_data['before_last_game_goals_scored'] + year_data['last_game_goals_scored']
                year_data['total_goals_conceded'] = year_data['before_last_game_goals_conceded'] + year_data['last_game_goals_conceded']
                year_data['total_goals_difference'] = year_data['total_goals_scored'] - year_data['total_goals_conceded']

                # Adjust points dynamically for each game outcome
                year_data['last_game_points'] = (
                    (year_data['last_game_goals_scored'] > year_data['last_game_goals_conceded']).astype(int) * 2 +
                    (year_data['last_game_goals_scored'] == year_data['last_game_goals_conceded']).astype(int) * 0 +
                    (year_data['last_game_goals_scored'] < year_data['last_game_goals_conceded']).astype(int) * -1
                )

                # Update total points
                year_data['total_points'] = year_data['before_last_game_points'] + year_data['last_game_points']

            # Recalculate third-placed team info for each group
            third_teams_info = {}

            for group_stage, group_data in year_data.groupby('stage'):
                group_key = group_stage.replace(" ", "_")  # Ensure valid keys

                # Sort teams by total points, goal difference, and goals scored
                sorted_standings = group_data.sort_values(
                    by=['total_points', 'total_goals_difference', 'total_goals_scored'], 
                    ascending=[False, False, False]
                )

                # Check for ties in total points
                tied_teams = sorted_standings[sorted_standings.duplicated(subset=['total_points'], keep=False)]

                if not tied_teams.empty:
                    # Resolve ties using tiebreaker_after
                    for i, row1 in tied_teams.iterrows():
                        for j, row2 in tied_teams.iterrows():
                            if i != j:  # Ensure no self-comparison
                                winner, _, _ = tiebreaker_after(row1, row2, agg_goals_before_last_day)
                                if winner == row1['team']:
                                    sorted_standings.loc[sorted_standings['team'] == row1['team'], 'tied_won'] += 1

                # Final sorting with tie-breaking criteria
                sorted_standings = sorted_standings.sort_values(
                    by=['total_points', 'tied_won', 'total_goals_difference', 'total_goals_scored'], 
                    ascending=[False, False, False, False]
                )

                # Extract third-placed team
                if len(sorted_standings) >= 3:
                    third_placed = sorted_standings.iloc[2]
                    third_teams_info[f'third_team_info_{group_key}'] = {
                        'team': third_placed['team'],
                        'points': int(third_placed['total_points']),
                        'goals_difference': int(third_placed['total_goals_difference']),
                        'goals_scored': int(third_placed['total_goals_scored'])
                    }
                else:
                    third_teams_info[f'third_team_info_{group_key}'] = None

            # Sort third-placed teams across groups and get top 2
            sorted_third_teams = sorted(
                [(info['team'], info['points'], info['goals_difference'], info['goals_scored']) for info in third_teams_info.values() if info],
                key=lambda x: (x[1], x[2], x[3]),
                reverse=True
            )
            top2_third_teams = [team[0] for team in sorted_third_teams[:2]]
            excluded_teams = [team[0] for team in sorted_third_teams[2:]]

            # Check for changes in top2_third_teams
            if top2_third_teams != previous_top2:
                changes += 1
                previous_top2 = top2_third_teams

            results.append({
                'year': year,
                'stage': stage,
                'goal_time': goal['goal_minute'],
                'home_team': home_team,
                'away_team': away_team,
                'scorer_team': player_team,
                **third_teams_info,
                'top2_third_teams': top2_third_teams,
                'excluded_teams': excluded_teams,
                'changes': changes
            })

    return pd.DataFrame(results)

## World Cup

### men

In [16]:
def best_four_third_placed_wc_men(goals_last_day_sorted, all_games_before_last, agg_goals_before_last_day):
    """
    Process goals and calculate standings for third-placed teams, returning stats after each goal,
    with a single column per group containing all third-team info. Adds a list of teams excluded from
    the best 4 third-placed teams after evaluating each goal and a changes variable that increments
    every time the composition of top4_third_teams changes.
    """
    # Filter data for years >= 2016
    all_games_before_last = all_games_before_last[(all_games_before_last['year'] >= 2016)].copy()
    all_games_before_last['year'] = all_games_before_last['year'].astype(int)

    # Add 1 point to each team for the assumed 0-0 starting score
    all_games_before_last['points'] += 1

    results = []

    # Group by year and process each year separately
    for year, year_data in all_games_before_last.groupby('year'):
        year = int(year)
        print(f"\n--- Processing Year: {year} ---")
        year_data = year_data.copy()

        # Initialize columns for tracking stats
        year_data['before_last_game_goals_scored'] = year_data['goals_scored']
        year_data['before_last_game_goals_conceded'] = year_data['goals_conceded']
        year_data['before_last_game_points'] = year_data['points']
        year_data['last_game_goals_scored'] = 0
        year_data['last_game_goals_conceded'] = 0
        year_data['total_goals_scored'] = year_data['before_last_game_goals_scored']
        year_data['total_goals_conceded'] = year_data['before_last_game_goals_conceded']
        year_data['total_goals_difference'] = year_data['total_goals_scored'] - year_data['total_goals_conceded']
        year_data['last_game_points'] = 0
        year_data['total_points'] = year_data['before_last_game_points']

        # Sort goals by time for the current year
        goals_last_day_year = goals_last_day_sorted[(goals_last_day_sorted['year'] == year)]

        # Initialize a set to track locked stages
        locked_stages = set()

        # Initial standings for goal_time = 0
        third_teams_info = {}
        year_data['tied_won'] = 0  # Initialize tied_won column

        # Sort teams by the `standing` variable
        sorted_standings = year_data.sort_values(by='standing', ascending=True)

        # Extract third-placed teams based on sorted order
        for group_stage, group_data in sorted_standings.groupby('stage'):
            group_key = group_stage.replace(" ", "_")  # Ensure valid keys
            if len(group_data) >= 3:
                third_placed = group_data.iloc[2]  # Third-placed team based on `standing`
                third_teams_info[f'third_team_info_{group_key}'] = {
                    'team': third_placed['team'],
                    'points': int(third_placed['total_points']),
                    'goals_difference': int(third_placed['total_goals_difference']),
                    'goals_scored': int(third_placed['total_goals_scored'])
                }
            else:
                third_teams_info[f'third_team_info_{group_key}'] = None

        # Determine the top 4 third-placed teams initially
        sorted_third_teams = sorted(
            [(info['team'], info['points'], info['goals_difference'], info['goals_scored']) for info in third_teams_info.values() if info],
            key=lambda x: (x[1], x[2], x[3]),
            reverse=True
        )
        top4_third_teams = [team[0] for team in sorted_third_teams[:4]]
        excluded_teams = [team[0] for team in sorted_third_teams[4:]]
        previous_top4 = top4_third_teams
        changes = 0

        # Add the initial state row for the year
        results.append({
            'year': year,
            'stage': None,
            'goal_time': 0,
            'home_team': None,
            'away_team': None,
            'scorer_team': None,
            **third_teams_info,
            'top4_third_teams': top4_third_teams,
            'excluded_teams': excluded_teams,
            'changes': changes
        })

        # Process goals
        for _, goal in goals_last_day_year.iterrows():
            stage = goal['stage']
            home_team = goal['home_team']
            away_team = goal['away_team']
            player_team = goal['scorer_nationality']

            # Lock stats for stages before the current one
            if stage not in locked_stages:
                for locked_stage in year_data['stage'].unique():
                    if locked_stage != stage and locked_stage not in locked_stages:
                        locked_stages.add(locked_stage)

            # Update scores for the current stage
            if stage not in locked_stages:
                opponent_team = home_team if player_team == away_team else away_team
                year_data.loc[year_data['team'] == player_team, 'last_game_goals_scored'] += 1
                year_data.loc[year_data['team'] == opponent_team, 'last_game_goals_conceded'] += 1

                # Update total goals scored, goals conceded, and goal difference
                year_data['total_goals_scored'] = year_data['before_last_game_goals_scored'] + year_data['last_game_goals_scored']
                year_data['total_goals_conceded'] = year_data['before_last_game_goals_conceded'] + year_data['last_game_goals_conceded']
                year_data['total_goals_difference'] = year_data['total_goals_scored'] - year_data['total_goals_conceded']

                # Adjust points dynamically for each game outcome
                year_data['last_game_points'] = (
                    (year_data['last_game_goals_scored'] > year_data['last_game_goals_conceded']).astype(int) * 2 +
                    (year_data['last_game_goals_scored'] == year_data['last_game_goals_conceded']).astype(int) * 0 +
                    (year_data['last_game_goals_scored'] < year_data['last_game_goals_conceded']).astype(int) * -1
                )

                # Update total points
                year_data['total_points'] = year_data['before_last_game_points'] + year_data['last_game_points']

            # Recalculate third-placed team info for each group
            third_teams_info = {}

            for group_stage, group_data in year_data.groupby('stage'):
                group_key = group_stage.replace(" ", "_")  # Ensure valid keys

                # Sort teams by total points, goal difference, and goals scored
                sorted_standings = group_data.sort_values(
                    by=['total_points', 'total_goals_difference', 'total_goals_scored'], 
                    ascending=[False, False, False]
                )

                # Check for ties in total points
                tied_teams = sorted_standings[sorted_standings.duplicated(subset=['total_points'], keep=False)]

                if not tied_teams.empty:
                    # Resolve ties using tiebreaker_after
                    for i, row1 in tied_teams.iterrows():
                        for j, row2 in tied_teams.iterrows():
                            if i != j:  # Ensure no self-comparison
                                winner, _, _ = tiebreaker_after(row1, row2, agg_goals_before_last_day)
                                if winner == row1['team']:
                                    sorted_standings.loc[sorted_standings['team'] == row1['team'], 'tied_won'] += 1

                # Final sorting with tie-breaking criteria
                sorted_standings = sorted_standings.sort_values(
                    by=['total_points', 'total_goals_difference', 'total_goals_scored', 'tied_won'], 
                    ascending=[False, False, False, False]
                )

                # Extract third-placed team
                if len(sorted_standings) >= 3:
                    third_placed = sorted_standings.iloc[2]
                    third_teams_info[f'third_team_info_{group_key}'] = {
                        'team': third_placed['team'],
                        'points': int(third_placed['total_points']),
                        'goals_difference': int(third_placed['total_goals_difference']),
                        'goals_scored': int(third_placed['total_goals_scored'])
                    }
                else:
                    third_teams_info[f'third_team_info_{group_key}'] = None

            # Sort third-placed teams across groups and get top 4
            sorted_third_teams = sorted(
                [(info['team'], info['points'], info['goals_difference'], info['goals_scored']) for info in third_teams_info.values() if info],
                key=lambda x: (x[1], x[2], x[3]),
                reverse=True
            )
            top4_third_teams = [team[0] for team in sorted_third_teams[:4]]
            excluded_teams = [team[0] for team in sorted_third_teams[4:]]

            # Check for changes in top4_third_teams
            if top4_third_teams != previous_top4:
                changes += 1
                previous_top4 = top4_third_teams

            results.append({
                'year': year,
                'stage': stage,
                'goal_time': goal['goal_minute'],
                'home_team': home_team,
                'away_team': away_team,
                'scorer_team': player_team,
                **third_teams_info,
                'top4_third_teams': top4_third_teams,
                'excluded_teams': excluded_teams,
                'changes': changes
            })

    return pd.DataFrame(results)

### women

In [17]:
def best_two_third_placed_wc_women(goals_last_day_sorted, all_games_before_last, agg_goals_before_last_day):
    """
    Process goals and calculate standings for third-placed teams, returning stats after each goal,
    with a single column per group containing all third-team info. Adds a list of teams excluded from
    the best 2 third-placed teams after evaluating each goal and a changes variable that increments
    every time the composition of top2_third_teams changes.
    """
    # Apply filter to process only years == 1991
    all_games_before_last = all_games_before_last[(all_games_before_last['year'] == 1991)].copy()  
    all_games_before_last['year'] = all_games_before_last['year'].astype(int)

    # Add 1 point to each team for the assumed 0-0 starting score
    all_games_before_last['points'] += 1

    results = []

    # Group by year and process each year separately
    for year, year_data in all_games_before_last.groupby('year'):
        year = int(year)
        print(f"\n--- Processing Year: {year} ---")
        year_data = year_data.copy()

        # Initialize columns for tracking stats
        year_data['before_last_game_goals_scored'] = year_data['goals_scored']
        year_data['before_last_game_goals_conceded'] = year_data['goals_conceded']
        year_data['before_last_game_points'] = year_data['points']
        year_data['last_game_goals_scored'] = 0
        year_data['last_game_goals_conceded'] = 0
        year_data['total_goals_scored'] = year_data['before_last_game_goals_scored']
        year_data['total_goals_conceded'] = year_data['before_last_game_goals_conceded']
        year_data['total_goals_difference'] = year_data['total_goals_scored'] - year_data['total_goals_conceded']
        year_data['last_game_points'] = 0
        year_data['total_points'] = year_data['before_last_game_points']

        # Sort goals by time for the current year
        goals_last_day_year = goals_last_day_sorted[(goals_last_day_sorted['year'] == year)]

        # Initialize a set to track locked stages
        locked_stages = set()

        # Initial standings for goal_time = 0
        third_teams_info = {}
        year_data['tied_won'] = 0  # Initialize tied_won column

        # Sort teams by the `standing` variable
        sorted_standings = year_data.sort_values(by='standing', ascending=True)

        # Extract third-placed teams based on sorted order
        for group_stage, group_data in sorted_standings.groupby('stage'):
            group_key = group_stage.replace(" ", "_")  # Ensure valid keys
            if len(group_data) >= 3:
                third_placed = group_data.iloc[2]  # Third-placed team based on `standing`
                third_teams_info[f'third_team_info_{group_key}'] = {
                    'team': third_placed['team'],
                    'points': int(third_placed['total_points']),
                    'goals_difference': int(third_placed['total_goals_difference']),
                    'goals_scored': int(third_placed['total_goals_scored'])
                }
            else:
                third_teams_info[f'third_team_info_{group_key}'] = None

        # Determine the top 2 third-placed teams initially
        sorted_third_teams = sorted(
            [(info['team'], info['points'], info['goals_difference'], info['goals_scored']) for info in third_teams_info.values() if info],
            key=lambda x: (x[1], x[2], x[3]),
            reverse=True
        )
        top2_third_teams = [team[0] for team in sorted_third_teams[:2]]
        excluded_teams = [team[0] for team in sorted_third_teams[2:]]
        previous_top2 = top2_third_teams
        changes = 0

        # Add the initial state row for the year
        results.append({
            'year': year,
            'stage': None,
            'goal_time': 0,
            'home_team': None,
            'away_team': None,
            'scorer_team': None,
            **third_teams_info,
            'top2_third_teams': top2_third_teams,
            'excluded_teams': excluded_teams,
            'changes': changes
        })

        # Process goals
        for _, goal in goals_last_day_year.iterrows():
            stage = goal['stage']
            home_team = goal['home_team']
            away_team = goal['away_team']
            player_team = goal['scorer_nationality']

            # Lock stats for stages before the current one
            if stage not in locked_stages:
                for locked_stage in year_data['stage'].unique():
                    if locked_stage != stage and locked_stage not in locked_stages:
                        locked_stages.add(locked_stage)

            # Update scores for the current stage
            if stage not in locked_stages:
                opponent_team = home_team if player_team == away_team else away_team
                year_data.loc[year_data['team'] == player_team, 'last_game_goals_scored'] += 1
                year_data.loc[year_data['team'] == opponent_team, 'last_game_goals_conceded'] += 1

                # Update total goals scored, goals conceded, and goal difference
                year_data['total_goals_scored'] = year_data['before_last_game_goals_scored'] + year_data['last_game_goals_scored']
                year_data['total_goals_conceded'] = year_data['before_last_game_goals_conceded'] + year_data['last_game_goals_conceded']
                year_data['total_goals_difference'] = year_data['total_goals_scored'] - year_data['total_goals_conceded']

                # Adjust points dynamically for each game outcome
                year_data['last_game_points'] = (
                    (year_data['last_game_goals_scored'] > year_data['last_game_goals_conceded']).astype(int) * 2 +
                    (year_data['last_game_goals_scored'] == year_data['last_game_goals_conceded']).astype(int) * 0 +
                    (year_data['last_game_goals_scored'] < year_data['last_game_goals_conceded']).astype(int) * -1
                )

                # Update total points
                year_data['total_points'] = year_data['before_last_game_points'] + year_data['last_game_points']

            # Recalculate third-placed team info for each group
            third_teams_info = {}

            for group_stage, group_data in year_data.groupby('stage'):
                group_key = group_stage.replace(" ", "_")  # Ensure valid keys

                # Sort teams by total points, goal difference, and goals scored
                sorted_standings = group_data.sort_values(
                    by=['total_points', 'total_goals_difference', 'total_goals_scored'], 
                    ascending=[False, False, False]
                )

                # Check for ties in total points
                tied_teams = sorted_standings[sorted_standings.duplicated(subset=['total_points'], keep=False)]

                if not tied_teams.empty:
                    # Resolve ties using tiebreaker_after
                    for i, row1 in tied_teams.iterrows():
                        for j, row2 in tied_teams.iterrows():
                            if i != j:  # Ensure no self-comparison
                                winner, _, _ = tiebreaker_after(row1, row2, agg_goals_before_last_day)
                                if winner == row1['team']:
                                    sorted_standings.loc[sorted_standings['team'] == row1['team'], 'tied_won'] += 1

                # Final sorting with tie-breaking criteria
                sorted_standings = sorted_standings.sort_values(
                    by=['total_points', 'tied_won', 'total_goals_difference', 'total_goals_scored'], 
                    ascending=[False, False, False, False]
                )

                # Extract third-placed team
                if len(sorted_standings) >= 3:
                    third_placed = sorted_standings.iloc[2]
                    third_teams_info[f'third_team_info_{group_key}'] = {
                        'team': third_placed['team'],
                        'points': int(third_placed['total_points']),
                        'goals_difference': int(third_placed['total_goals_difference']),
                        'goals_scored': int(third_placed['total_goals_scored'])
                    }
                else:
                    third_teams_info[f'third_team_info_{group_key}'] = None

            # Sort third-placed teams across groups and get top 2
            sorted_third_teams = sorted(
                [(info['team'], info['points'], info['goals_difference'], info['goals_scored']) for info in third_teams_info.values() if info],
                key=lambda x: (x[1], x[2], x[3]),
                reverse=True
            )
            top2_third_teams = [team[0] for team in sorted_third_teams[:2]]
            excluded_teams = [team[0] for team in sorted_third_teams[2:]]

            # Check for changes in top2_third_teams
            if top2_third_teams != previous_top2:
                changes += 1
                previous_top2 = top2_third_teams

            results.append({
                'year': year,
                'stage': stage,
                'goal_time': goal['goal_minute'],
                'home_team': home_team,
                'away_team': away_team,
                'scorer_team': player_team,
                **third_teams_info,
                'top2_third_teams': top2_third_teams,
                'excluded_teams': excluded_teams,
                'changes': changes
            })

    return pd.DataFrame(results)